In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn import neighbors, datasets, svm
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

import matplotlib
#matplotlib.use('GTKAgg')
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap
%matplotlib inline
matplotlib.rcParams.update({'font.size': 18})

Support vector Machine (SVM) algorithm creates a line or a hyperplace which separates the data into classes. 

In [2]:
df = pd.read_excel ('data/All Data combined.xlsx')

<ipython-input-2-36f5b1f1bff7>:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  df = pd.read_excel ('data/All Data combined.xlsx')


In [16]:
X = df[['number of trials','accuracy [calc. max/ actual max]','time per trial [s]']]
y = df['type_of_opt']

print(len(y))

1500


In [17]:
# split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=14) #14 is seed

print(len(y_train), len(y_test))

1200 300


In [9]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(svm.SVC(gamma='auto'), {
    'C': [700,800,900,1000,1100,1200,1300],
    'kernel':['rbf','linear']
}, cv=5, return_train_score=True)

clf.fit(X,df['type_of_opt'])
clf.cv_results_

{'mean_fit_time': array([ 0.13582444, 10.46857543,  0.1405334 , 11.1163065 ,  0.14903736,
        11.97170119,  0.14297781, 12.31560469,  0.16611037, 14.33282175,
         0.16724892, 15.40260129,  0.18066616, 15.3027514 ]),
 'std_fit_time': array([0.0116499 , 0.7878376 , 0.01071964, 1.74418747, 0.01676827,
        1.02258521, 0.01442362, 1.23300106, 0.01684041, 0.64861898,
        0.01715262, 1.1608674 , 0.01643952, 1.6129455 ]),
 'mean_score_time': array([0.02484527, 0.00756669, 0.02519393, 0.00744367, 0.02397609,
        0.00719914, 0.02293544, 0.00724764, 0.025734  , 0.00775943,
        0.02617965, 0.00800576, 0.02546148, 0.00767121]),
 'std_score_time': array([9.65341636e-04, 8.09658500e-04, 2.16478174e-03, 3.26708381e-04,
        1.17183230e-03, 8.11023613e-05, 5.76599911e-04, 5.13514156e-04,
        1.05021666e-03, 5.64275453e-04, 1.14756179e-03, 2.53154486e-04,
        3.52298781e-04, 2.47042360e-04]),
 'param_C': masked_array(data=[700, 700, 800, 800, 900, 900, 1000, 1000, 110

In [10]:
clf.best_score_

0.6466666666666667

In [11]:
clf.best_params_

{'C': 1300, 'kernel': 'linear'}

In [32]:
model = svm.SVC(kernel='linear',C=3000,gamma='auto')
model.fit(X_train,y_train)
model.score(X_test, y_test)
y_1_test_predicted = model.predict(X_test)


In [25]:
model.score(X_test, y_test)

0.7366666666666667

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(svm.SVC(gamma='auto'), {
        'C': [1000,1500,2000,2500],
        'kernel':['rbf','linear']
    },
    cv=5, #cross validation
    return_train_score=False,
    n_iter=21
)
rs.fit(X, df['type_of_opt'])
pd.DataFrame(rs.cv_results_)[['param_C','param_kernel','mean_test_score']]

/Users/saleksegid/miniconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:285: UserWarning: The total space of parameters 8 is smaller than n_iter=21. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [5]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

comparing different models

In [6]:
model_params = {
    'svm':{
        'model': svm.SVC(gamma='auto'),
        'params': {
            'C': [50,100,500],
            'kernel': ['rbf','linear']
        }
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [0,5,10]
        }
    },
    'logistic_regression': {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [50,100,500]
        }
    }
}

In [7]:
from sklearn.model_selection import GridSearchCV

In [8]:
scores = []

for model_name, mp in model_params.items():
    clf = GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(X, df['type_of_opt'])
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })

/Users/saleksegid/miniconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/saleksegid/miniconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/saleksegid/miniconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 349, in fit
    self._validate_estimator()
  File "/Users/saleksegid/miniconda3/lib/python3.8/site-packages/sklearn/ensemble/_base.py", line 133, in _validate_estimator
    raise ValueError("n_estimators must be greater than zero, "
ValueError: n_estimators must be greater than zero, got 0.

  warnings.warn("Estimator fit failed. The score on this train-test"
/Users/saleksegid/miniconda3/lib/python3.8/site-packages/sklearn/model_selection/_valida

In [9]:
df_2 = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df_2

,model,best_score,best_params
0,svm,0.636667,"{'C': 500, 'kernel': 'linear'}"
1,random_forest,0.659333,{'n_estimators': 10}
2,logistic_regression,0.541333,{'C': 500}


In [18]:
models = (svm.SVC(kernel='linear', C=1000),
        svm.LinearSVC(C=1000, max_iter=100),
         svm.SVC(kernel='rbf', gamma=0.8, C=1000),
         svm.SVC(kernel='poly', degree=3, gamma='auto', C=1000))

models = (clf.fit(X_train, y_train) for clf in models)

In [21]:
y_predicted = clf.predict(X_test)
y_predicted

array(['Bayes_s', 'Random', 'Random', 'Random', 'TPE', 'Random', 'Random',
       'CmaEs', 'CmaEs', 'CmaEs', 'Bayes_s', 'Random', 'CmaEs', 'Bayes_s',
       'CmaEs', 'CmaEs', 'Bayes_B', 'Bayes_B', 'CmaEs', 'CmaEs',
       'Bayes_B', 'Random', 'Bayes_s', 'Random', 'TPE', 'CmaEs', 'Random',
       'Random', 'Random', 'Random', 'Bayes_s', 'Bayes_B', 'Random',
       'Bayes_s', 'Bayes_s', 'Bayes_s', 'CmaEs', 'Bayes_s', 'Bayes_B',
       'Random', 'Bayes_B', 'CmaEs', 'Bayes_B', 'Random', 'Random',
       'CmaEs', 'Bayes_s', 'Random', 'Bayes_s', 'Random', 'CmaEs',
       'Bayes_s', 'Bayes_s', 'Bayes_s', 'CmaEs', 'CmaEs', 'CmaEs', 'TPE',
       'Bayes_s', 'Bayes_s', 'Bayes_s', 'Random', 'TPE', 'Bayes_s',
       'Bayes_B', 'Bayes_B', 'Random', 'TPE', 'Bayes_s', 'CmaEs', 'TPE',
       'Bayes_s', 'Bayes_B', 'Random', 'Bayes_s', 'Bayes_B', 'Random',
       'Bayes_s', 'CmaEs', 'Bayes_s', 'Random', 'Bayes_s', 'Bayes_B',
       'CmaEs', 'Bayes_B', 'Bayes_B', 'Bayes_B', 'Bayes_s', 'CmaEs',
       'Ba